## Understanding the E91 Quantum Key Distribution Protocol

Quantum Key Distribution (QKD) ensures secure communication by using the principles of quantum mechanics. One of the most well-known QKD protocols is *E91, proposed by Artur Ekert in 1991. It leverages **quantum entanglement* and *Bell's theorem* to ensure that any attempt at eavesdropping can be detected.

### What is the E91 Protocol?

The *E91 protocol* uses *entangled quantum particles* to allow two parties (commonly known as Alice and Bob) to generate a shared, secret cryptographic key. It differs from earlier QKD protocols like BB84 by using *entanglement* instead of individually prepared quantum states.

The security of E91 is guaranteed by the *CHSH inequality* — a testable condition from quantum mechanics. Any deviation from the expected quantum correlations signals potential *eavesdropping*.

<div align="center">
  <img src="e91_diagram.png" width="500"/>
  <p><em>Figure: Diagrammatic view of the E91 protocol showing entangled photon transmission to Alice and Bob.</em></p>
</div>

### How the E91 Protocol Works: Step-by-Step (Aligned with the Diagram)

1. *Entanglement Source:*
   - A *quantum entangled photon source* (shown in the center of the diagram) emits entangled photon pairs in the Bell state:
     $$
     |\Phi^+\rangle = \frac{1}{\sqrt{2}} (|00\rangle + |11\rangle)
     $$
   - One photon from each pair is sent to *Alice* (left side of the image), and the other to *Bob* (right side).

2. *Random Basis Selection:*
   - *Alice* and *Bob* each randomly select one of *three measurement angles*:
     - Alice uses measurement angles:  
       $$
       \phi^a_i = \left\{0, \frac{\pi}{4}, \frac{\pi}{2} \right\}
       $$
     - Bob uses measurement angles:  
       $$
       \phi^b_j = \left\{ \frac{\pi}{4}, \frac{\pi}{2}, \frac{3\pi}{4} \right\}
       $$
   - These correspond to quantum observables constructed from Pauli matrices:
     $$
     W = \frac{X + Z}{\sqrt{2}}, \quad V = \frac{-X + Z}{\sqrt{2}}
     $$

3. *Quantum Measurement:*
   - Each photon is measured along the chosen direction (represented by the lines going to detectors).
   - Outcomes are either *+1 or -1*, corresponding to spin or polarization directions.

4. *Basis Announcement:*
   - After all measurements are done, Alice and Bob *publicly announce* which angles (measurement bases) they used — *not the results*.
   - This step is reflected after the detectors and leads into *information processing (IP)*.

5. *Key Sifting:*
   - Only outcomes where their chosen bases are *compatible* (i.e., expected to be strongly correlated) are kept to form the *raw key*.
   - This decision is taken in the *IP (Information Processing)* block in the diagram.

6. *CHSH Inequality Test (Security Check):*
   - A subset of results (where different bases were used) is used to compute the *CHSH parameter*:
     $$
     \langle S \rangle = \langle Z \otimes W \rangle + \langle Z \otimes V \rangle + \langle X \otimes W \rangle - \langle X \otimes V \rangle
     $$
     - *Classical limit:*
       $$
       |\langle S \rangle| \leq 2
       $$
     - *Quantum prediction:*
       $$
       |\langle S \rangle| = 2\sqrt{2}
       $$
   - If the observed value exceeds 2, it confirms *entanglement* and the *absence of eavesdropping*.
   - This step too is part of the *IP block*, shown at the bottom.

7. *Error Correction & Privacy Amplification:*
   - The IP block performs *error correction* to fix mismatches and *privacy amplification* to remove any leaked information.
   - The final output is a *shared, secret key* — secure even in the presence of potential eavesdropping.

In [ ]:
# E91 Quantum Key Distribution Simulator
# Comprehensive simulation framework for E91 protocol over different channel types

# ========================== Import Libraries ==========================
import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin, pi, sqrt, log2, exp
from enum import Enum
from typing import Tuple, Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib style for better plots
plt.style.use('seaborn-v0_8' if 'seaborn-v0_8' in plt.style.available else 'default')

# 2.2 Channel Class

- Models a quantum channel: Optical Fiber or Free Space Optical (FSO)  

- *Fiber channel efficiency* decreases exponentially with distance:  
  $$ \eta = \eta_0 \cdot 10^{- \alpha L / 10} $$
  where $\alpha$ is attenuation (dB/km), and \( L \) is distance (km)

- *FSO channel efficiency* includes:
  - Geometric loss:  
    $$ \left( \frac{d_r}{d_t + \theta L} \right)^2 $$  
  - Atmospheric loss:  
    $$ \exp(-\alpha L) $$  
  - Combined:  
    $$ \eta = \eta_0 \cdot \left( \frac{d_r}{d_t + \theta L} \right)^2 \cdot \exp(-\alpha L) $$

- Misalignment error increases linearly with distance  
- Stray light included for FSO: $$ p_{stray} = 5 \times 10^{-6} $$ 
- Photon transmission modeled via binomial distribution based on channel efficiency

In [ ]:
# ========================== Channel Class ==========================
class ChannelType(Enum):
    FIBER = "fiber"
    FSO = "fso"  # Free Space Optical

class QuantumChannel:
    """
    Quantum channel class supporting both fiber and FSO transmission
    """
    
    def __init__(self, channel_type: ChannelType):
        self.channel_type = channel_type
        self._set_channel_parameters()
    
    def _set_channel_parameters(self):
        """Set channel-specific parameters"""
        if self.channel_type == ChannelType.FIBER:
            # Fiber channel parameters
            self.alpha_db = 0.2            # Atmospheric attenuation (dB/km)
            self.max_distance = 300        # Maximum practical distance (km)
            self.noise_components = {
                'raman': 5e-5,             # Raman scattering noise
                'dark': 5e-6               # Dark count noise
            }
        
        elif self.channel_type == ChannelType.FSO:
            # Free Space Optical parameters
            self.alpha_db = 0.1            # Atmospheric attenuation (dB/km)
            self.max_distance = 50         # Maximum practical distance (km)
            self.beam_divergence = 0.025e-3  # Beam divergence (rad)
            self.tx_aperture = 0.01        # Transmitter aperture diameter (m)
            self.rx_aperture = 0.03        # Receiver aperture diameter (m)
            self.noise_components = {
                'stray': 5e-6,             # Stray light noise
                'dark': 5e-6               # Dark count noise
            }
        
        # Convert dB/km to 1/km for calculations
        self.alpha_linear = self.alpha_db / 4.343
        self.total_noise_prob = sum(self.noise_components.values())
    
    def transmittance(self, distance_km: float) -> float:
        """
        Calculate channel transmittance for given distance
        
        Args:
            distance_km: Distance in kilometers
            
        Returns:
            Transmittance value between 0 and 1
        """
        if self.channel_type == ChannelType.FIBER:
            # Fiber: Only atmospheric/material losses
            return 10 ** (-self.alpha_db * distance_km / 10)
        
        elif self.channel_type == ChannelType.FSO:
            # FSO: Geometric spreading + atmospheric losses
            distance_m = distance_km * 1000
            
            # Geometric loss due to beam divergence
            geo_loss = (self.rx_aperture / (self.tx_aperture + self.beam_divergence * distance_m)) ** 2
            
            # Atmospheric loss
            atm_loss = exp(-self.alpha_linear * distance_km)
            
            return geo_loss * atm_loss
    
    def get_channel_info(self) -> Dict:
        """Return channel configuration information"""
        return {
            'type': self.channel_type.value,
            'alpha_db': self.alpha_db,
            'max_distance': self.max_distance,
            'noise_components': self.noise_components,
            'total_noise': self.total_noise_prob
        }
    
    def update_parameters(self, **kwargs):
        """Update channel parameters dynamically"""
        for key, value in kwargs.items():
            if hasattr(self, key):
                setattr(self, key, value)
                if key == 'alpha_db':
                    self.alpha_linear = value / 4.343
                elif key in ['raman', 'dark', 'stray']:
                    self.noise_components[key] = value
                    self.total_noise_prob = sum(self.noise_components.values())

---

## 2. Quantum Detection Theory

### Understanding Photon Detection in QKD

In quantum key distribution, the detection process is probabilistic and affected by several factors:

#### Detection Efficiency Components

1. **Detector Efficiency** ($\eta_{det}$): Probability that a photon hitting the detector is converted to an electrical signal
   - Typical values: 60-90% for modern detectors (APDs, SPADs, SNSPDs)
   - Depends on wavelength, temperature, and detector type

2. **Collection Efficiency** ($\eta_{col}$): Probability that a photon entering the system reaches the detector
   - Includes fiber coupling losses, beam splitter losses, filter losses
   - Typical values: 50-80%

3. **Total Detection Efficiency**: $\eta_{total} = \eta_{det} \times \eta_{col}$

#### Photon Number States After Transmission

For an entangled photon pair with channel transmittance $T$:

- **Both photons arrive**: $P_{both} = T^2$
- **One photon arrives**: $P_{single} = 2T(1-T)$  
- **No photons arrive**: $P_{none} = (1-T)^2$

These probabilities are crucial for understanding coincidence detection in E91.

#### Click Probability Model

The total click probability includes both signal and noise:
$$P_{click} = \eta_{total} + 2P_{noise}(1 - \eta_{total})$$

#### Normalization Factor for Correlations

The normalization factor $N$ accounts for detection inefficiencies and noise:

$$N = \frac{P_{both} \cdot \eta_{total}^2}{P_{both} \cdot P_{click}^2 + 2P_{single} \cdot P_{noise} \cdot P_{click} + 4P_{none} \cdot P_{noise}^2}$$

This factor ensures that measured correlations properly reflect the underlying quantum entanglement despite imperfect detection.

#### Detector Types in QKD

1. **Avalanche Photodiodes (APDs)**: Common, moderate efficiency (~60-80%)
2. **Single Photon Avalanche Diodes (SPADs)**: Good timing resolution
3. **Superconducting Nanowire Single Photon Detectors (SNSPDs)**: High efficiency (~90%+), low noise

In [ ]:
# ========================== Detector Class ==========================
class QuantumDetector:
    """
    Quantum detector system for photon pair detection
    """
    
    def __init__(self, detector_efficiency: float = 0.6, collection_efficiency: float = 0.6,
             pair_generation_rate: float = 0.64e6):
            self.eta_detector = detector_efficiency
            self.eta_collection = collection_efficiency
            self.eta_total = self.eta_detector * self.eta_collection
            self.pair_generation_rate = pair_generation_rate

        
    def compute_detection_probabilities(self, transmittance: float) -> Dict[str, float]:
        """
        Compute detection probabilities for different photon number states
        
        Args:
            transmittance: Channel transmittance
            
        Returns:
            Dictionary with probabilities for different detection scenarios
        """
        T = transmittance
        
        # Photon number state probabilities after transmission
        p_both = T ** 2          # Both photons arrive
        p_single = 2 * T * (1 - T)  # One photon arrives
        p_none = (1 - T) ** 2    # No photons arrive
        
        return {
            'p_both_arrive': p_both,
            'p_single_arrive': p_single,
            'p_none_arrive': p_none
        }
    
    def compute_click_probability(self, noise_prob: float) -> float:
        """
        Compute probability of detector click (including noise)
        
        Args:
            noise_prob: Total noise probability per detector
            
        Returns:
            Click probability
        """
        return self.eta_total + 2 * noise_prob * (1 - self.eta_total)
    
    def compute_normalization_factor(self, transmittance: float, noise_prob: float) -> float:
        """
        Compute normalization factor N for correlation calculations
        
        Args:
            transmittance: Channel transmittance
            noise_prob: Total noise probability
            
        Returns:
            Normalization factor N
        """
        probs = self.compute_detection_probabilities(transmittance)
        click_prob = self.compute_click_probability(noise_prob)
        
        # Numerator: Both photons detected successfully
        numerator = probs['p_both_arrive'] * (self.eta_total ** 2)
        
        # Denominator: All possible click combinations
        denominator = (probs['p_both_arrive'] * (click_prob ** 2) +
                      2 * probs['p_single_arrive'] * noise_prob * click_prob +
                      4 * probs['p_none_arrive'] * (noise_prob ** 2))
        
        return numerator / denominator if denominator > 0 else 0
    
    def update_efficiency(self, detector_eff: Optional[float] = None, 
                         collection_eff: Optional[float] = None):
        """Update detector efficiencies"""
        if detector_eff is not None:
            self.eta_detector = detector_eff
        if collection_eff is not None:
            self.eta_collection = collection_eff
        self.eta_total = self.eta_detector * self.eta_collection



## E91 QKD Simulator – Physics-Informed Core

This section defines the `E91Simulator` class, which provides a complete simulation of the E91 entanglement-based quantum key distribution protocol. The simulator supports realistic modeling of both fiber and free-space optical (FSO) channels, including key physical effects.

### Key Features

- Bell inequality (CHSH) violation analysis using measurement correlations
- Secret Key Rate (SKR) estimation using the Acín et al. entropic bound
- Quantum Bit Error Rate (QBER) calculation from Bell parameters
- Support for center-source architecture: the entangled photon source is equidistant from Alice and Bob
- Integration of physics-based effects:
  - Channel transmittance models (fiber and FSO)
  - Detector efficiency with saturation effects
  - Afterpulsing probability
  - Timing jitter errors
  - Distance-dependent misalignment

### Class Structure Overview

- `simulate_single_distance(distance_km)`  
  Simulates protocol performance for a fixed transmission distance

- `simulate_distance_range(min_distance, max_distance, num_points)`  
  Computes SKR and QBER across a specified range of distances

- `compute_bell_parameter(normalization, phase)`  
  Computes the Bell parameter \( S \) using correlation functions

- `compute_secret_key_rate(S, QBER, T)`  
  Estimates the secret key rate in bits per second

### Usage

Before using the simulator, instantiate the required components:

```python
channel = QuantumChannel(ChannelType.FIBER)  # or ChannelType.FSO
detector = QuantumDetector()
simulator = E91Simulator(channel, detector, distance_km=40)


In [ ]:
# ========================== E91 Simulator Class ==========================
class E91Simulator:
    """
    Complete E91 quantum key distribution simulator
    """
    
    def __init__(self, channel: QuantumChannel, detector: QuantumDetector):
        self.channel = channel
        self.detector = detector
        
        # Bell test measurement settings (CHSH inequality)
        self.measurement_angles = {
            'alice_1': 0,           # θ₁ᴬ
            'alice_2': pi / 4,      # θ₃ᴬ  
            'bob_1': -pi / 8,       # θ₁ᴮ
            'bob_2': pi / 8         # θ₃ᴮ
        }
        
        # Entangled state phase (maximally entangled)
        self.entanglement_phase = pi
        
        # Results storage
        self.simulation_results = {}
    
    @staticmethod
    def binary_entropy(x: float) -> float:
        """
        Calculate binary entropy function H(x) = -x*log₂(x) - (1-x)*log₂(1-x)
        
        Args:
            x: Probability value between 0 and 1
            
        Returns:
            Binary entropy value
        """
        epsilon = 1e-12
        x = np.clip(x, epsilon, 1 - epsilon)
        return -x * log2(x) - (1 - x) * log2(1 - x)
    
    def correlation_function(self, theta_alice: float, theta_bob: float, 
                           normalization: float, phase: float) -> float:
        """
        Calculate correlation function E(θₐ, θᵦ) for given measurement angles
        
        Args:
            theta_alice: Alice's measurement angle
            theta_bob: Bob's measurement angle
            normalization: Normalization factor N
            phase: Entanglement phase
            
        Returns:
            Correlation value
        """
        term1 = -cos(2 * theta_alice) * cos(2 * theta_bob)
        term2 = cos(phase) * sin(2 * theta_alice) * sin(2 * theta_bob)
        return normalization * (term1 + term2)
    
    def compute_bell_parameter(self, normalization: float, phase: float) -> float:
        """
        Compute Bell parameter S for CHSH inequality
        
        Args:
            normalization: Normalization factor N
            phase: Entanglement phase
            
        Returns:
            Bell parameter S
        """
        angles = self.measurement_angles
        
        # Calculate four correlation functions
        E11 = self.correlation_function(angles['alice_1'], angles['bob_1'], 
                                       normalization, phase)
        E12 = self.correlation_function(angles['alice_1'], angles['bob_2'], 
                                       normalization, phase)
        E21 = self.correlation_function(angles['alice_2'], angles['bob_1'], 
                                       normalization, phase)
        E22 = self.correlation_function(angles['alice_2'], angles['bob_2'], 
                                       normalization, phase)
        
        # CHSH combination: S = |E₁₁ + E₁₂ - E₂₁ + E₂₂|
        S = abs(E11 + E12 - E21 + E22)
        
        return S
    
    def compute_qber(self, bell_parameter: float) -> float:
        """
        Compute Quantum Bit Error Rate (QBER) from Bell parameter
        
        Args:
            bell_parameter: Bell parameter S
            
        Returns:
            QBER value
        """
        # Clip S to valid range [0, 2√2]
        S = np.clip(bell_parameter, 0, 2 * sqrt(2))
        return 0.5 * (1 - S / (2 * sqrt(2)))
    
    def compute_secret_key_rate(self, bell_parameter: float, qber: float, 
                              transmittance: float) -> float:
        """
        Compute Secret Key Rate using Acín et al.'s formula
        
        Args:
            bell_parameter: Bell parameter S
            qber: Quantum bit error rate
            transmittance: Channel transmittance
            
        Returns:
            Secret key rate in bits per second
        """
        S = bell_parameter
        
        # No key extraction possible if S ≤ 2 (no Bell violation)
        if S <= 2:
            return 0
        
        # Acín et al. formula term
        term = (1 + sqrt(S**2 / 4 - 1)) / 2
        
        # SKR = (1/3) * ν * T * [1 - H(Q) - H(term)]
        skr = (1/3) * self.detector.pair_generation_rate * transmittance * \
              (1 - self.binary_entropy(qber) - self.binary_entropy(term))
        
        return max(0, skr)  # Ensure non-negative
    
    def simulate_single_distance(self, distance_km: float) -> Dict:
        """
        Simulate E91 protocol for a single distance
        
        Args:
            distance_km: Distance in kilometers
            
        Returns:
            Dictionary with simulation results
        """
        # Calculate channel transmittance
        T = self.channel.transmittance(distance_km)
        
        # Compute normalization factor
        N = self.detector.compute_normalization_factor(T, self.channel.total_noise_prob)
        
        # Calculate Bell parameter
        S = self.compute_bell_parameter(N, self.entanglement_phase)
        
        # Calculate QBER
        Q = self.compute_qber(S)
        
        # Calculate Secret Key Rate
        SKR = self.compute_secret_key_rate(S, Q, T)
        
        return {
            'distance': distance_km,
            'transmittance': T,
            'normalization': N,
            'bell_parameter': S,
            'qber': Q,
            'qber_percent': Q * 100,
            'secret_key_rate': SKR,
            'bell_violation': S > 2,
            'secure_communication': S > 2 and Q < 0.146  # 14.6% threshold
        }
    
    def simulate_distance_range(self, min_distance: float = 0.01, 
                              max_distance: Optional[float] = None,
                              num_points: int = 300) -> Dict:
        """
        Simulate E91 protocol over a range of distances
        
        Args:
            min_distance: Minimum distance in km
            max_distance: Maximum distance in km (uses channel max if None)
            num_points: Number of distance points to simulate
            
        Returns:
            Dictionary with arrays of simulation results
        """
        if max_distance is None:
            max_distance = self.channel.max_distance
        
        distances = np.linspace(min_distance, max_distance, num_points)
        
        results = {
            'distances': distances,
            'transmittances': [],
            'bell_parameters': [],
            'qbers': [],
            'qber_percents': [],
            'secret_key_rates': [],
            'bell_violations': [],
            'secure_regions': []
        }
        
        for distance in distances:
            single_result = self.simulate_single_distance(distance)
            
            results['transmittances'].append(single_result['transmittance'])
            results['bell_parameters'].append(single_result['bell_parameter'])
            results['qbers'].append(single_result['qber'])
            results['qber_percents'].append(single_result['qber_percent'])
            results['secret_key_rates'].append(single_result['secret_key_rate'])
            results['bell_violations'].append(single_result['bell_violation'])
            results['secure_regions'].append(single_result['secure_communication'])
        
        # Convert lists to numpy arrays for easier manipulation
        for key in results:
            if key != 'distances':
                results[key] = np.array(results[key])
        
        # Store results for later use
        self.simulation_results = results
        
        return results
    
    def update_measurement_angles(self, **angles):
        """
        Update measurement angles for Bell test
        
        Args:
            **angles: Keyword arguments for angle updates
                     (alice_1, alice_2, bob_1, bob_2)
        """
        for key, value in angles.items():
            if key in self.measurement_angles:
                self.measurement_angles[key] = value
    
    def get_simulation_summary(self) -> Dict:
        """Get summary of current simulation setup"""
        return {
            'channel_info': self.channel.get_channel_info(),
            'detector_efficiency': self.detector.eta_total,
            'measurement_angles': self.measurement_angles,
            'entanglement_phase': self.entanglement_phase
        }

# ========================== Example Usage ==========================
print("E91 QKD Simulator Classes Initialized Successfully!")
print("\nAvailable Classes:")
print("- QuantumChannel: Models fiber and FSO channels")
print("- QuantumDetector: Simulates photon detection systems") 
print("- E91Simulator: Complete E91 protocol simulation")
print("\nExample instantiation:")
print("channel = QuantumChannel(ChannelType.FIBER)")
print("detector = QuantumDetector()")
print("simulator = E91Simulator(channel, detector)")
print("\nRealistic Physics Features:")
print("✓ Center-source configuration")
print("✓ Detector afterpulsing and timing jitter")
print("✓ Distance-dependent misalignment errors")
print("✓ Saturation effects for multi-photon detection")
print("✓ Bounded QBER (≤ 50% physical maximum)")

In [ ]:
# ========================== Plotting QBER vs Distance ==========================
import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin, pi, sqrt, log2, exp
from enum import Enum
from typing import Tuple, Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib style for better plots
plt.style.use('seaborn-v0_8' if 'seaborn-v0_8' in plt.style.available else 'default')

# Assuming the classes from your original code are imported/defined above this point
# [Your E91 simulator classes would be here]

# Initialize channels and detector
fiber_channel = QuantumChannel(ChannelType.FIBER)
fso_channel = QuantumChannel(ChannelType.FSO)
detector = QuantumDetector()

# Initialize simulators (removed the invalid distance_km parameter)
fiber_simulator = E91Simulator(fiber_channel, detector)
fso_simulator = E91Simulator(fso_channel, detector)

# Run simulations with custom distance ranges
# Fiber can go longer distances, FSO is limited
fiber_results = fiber_simulator.simulate_distance_range(min_distance=0.1, max_distance=200, num_points=300)
fso_results = fso_simulator.simulate_distance_range(min_distance=0.1, max_distance=50, num_points=300)

# Create comprehensive plotting
plt.figure(figsize=(15, 10))

# Plot 1: QBER vs Distance
plt.subplot(2, 2, 1)
plt.plot(fiber_results['distances'], fiber_results['qber_percents'], 
         label='Fiber Channel', color='blue', linewidth=2)
plt.plot(fso_results['distances'], fso_results['qber_percents'], 
         label='FSO Channel', color='green', linewidth=2)
plt.axhline(y=14.6, color='red', linestyle='--', alpha=0.7, label='Security Threshold (14.6%)')
plt.xlabel('Distance (km)')
plt.ylabel('QBER (%)')
plt.title('QBER vs Distance Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 50)  # Cap at 50% (theoretical maximum)

# Plot 2: Bell Parameter vs Distance
plt.subplot(2, 2, 2)
plt.plot(fiber_results['distances'], fiber_results['bell_parameters'], 
         label='Fiber Channel', color='blue', linewidth=2)
plt.plot(fso_results['distances'], fso_results['bell_parameters'], 
         label='FSO Channel', color='green', linewidth=2)
plt.axhline(y=2, color='red', linestyle='--', alpha=0.7, label='Classical Limit (S=2)')
plt.axhline(y=2*sqrt(2), color='orange', linestyle='--', alpha=0.7, label='Quantum Limit (S=2√2)')
plt.xlabel('Distance (km)')
plt.ylabel('Bell Parameter S')
plt.title('Bell Parameter vs Distance')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Secret Key Rate vs Distance
plt.subplot(2, 2, 3)
plt.semilogy(fiber_results['distances'], fiber_results['secret_key_rates'], 
             label='Fiber Channel', color='blue', linewidth=2)
plt.semilogy(fso_results['distances'], fso_results['secret_key_rates'], 
             label='FSO Channel', color='green', linewidth=2)
plt.xlabel('Distance (km)')
plt.ylabel('Secret Key Rate (bits/s)')
plt.title('Secret Key Rate vs Distance (Log Scale)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 4: Channel Transmittance vs Distance
plt.subplot(2, 2, 4)
plt.semilogy(fiber_results['distances'], fiber_results['transmittances'], 
             label='Fiber Channel', color='blue', linewidth=2)
plt.semilogy(fso_results['distances'], fso_results['transmittances'], 
             label='FSO Channel', color='green', linewidth=2)
plt.xlabel('Distance (km)')
plt.ylabel('Transmittance')
plt.title('Channel Transmittance vs Distance (Log Scale)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print some key results
print("=== E91 QKD Simulation Results ===")
print(f"\nFiber Channel:")
print(f"  Max distance simulated: {fiber_results['distances'][-1]:.1f} km")
print(f"  QBER at max distance: {fiber_results['qber_percents'][-1]:.2f}%")
print(f"  Bell parameter at max distance: {fiber_results['bell_parameters'][-1]:.3f}")

print(f"\nFSO Channel:")
print(f"  Max distance simulated: {fso_results['distances'][-1]:.1f} km")
print(f"  QBER at max distance: {fso_results['qber_percents'][-1]:.2f}%")
print(f"  Bell parameter at max distance: {fso_results['bell_parameters'][-1]:.3f}")

# Find maximum secure distances
fiber_secure_distances = fiber_results['distances'][fiber_results['secure_regions']]
fso_secure_distances = fso_results['distances'][fso_results['secure_regions']]

if len(fiber_secure_distances) > 0:
    print(f"\nFiber secure range: up to {fiber_secure_distances[-1]:.1f} km")
else:
    print("\nFiber: No secure communication possible")

if len(fso_secure_distances) > 0:
    print(f"FSO secure range: up to {fso_secure_distances[-1]:.1f} km")
else:
    print("FSO: No secure communication possible")

In [ ]:
def plot_qber_vs_distance_e91(simulator_class, channel_type="fiber", 
                               detector_kwargs=None, distance_range=(0, 100), 
                               points=100):
    if detector_kwargs is None:
        detector_kwargs = {}

    channel = QuantumChannel(ChannelType(channel_type))
    detector = QuantumDetector(**detector_kwargs)
    simulator = simulator_class(channel, detector, distance_km=0)

    results = simulator.simulate_distance_range(
        min_distance=distance_range[0],
        max_distance=distance_range[1],
        num_points=points
    )

    plt.figure(figsize=(10, 6))
    plt.plot(results['distances'], results['qber_percents'], 'b-', label='QBER (%)')
    plt.axhline(y=11, color='r', linestyle='--', label='QBER 11% Threshold')
    plt.axhline(y=5, color='magenta', linestyle='--', label='QBER 5% Threshold')
    plt.grid(True)
    plt.xlabel('Distance (km)', fontsize=18)
    plt.ylabel('QBER (%)', fontsize=18)
    plt.title(f'E91: QBER vs Distance ({channel_type.upper()})', fontsize=20)
    plt.legend(fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.show()


In [ ]:
def plot_skr_vs_distance_e91(simulator_class, channel_type="fiber", 
                             detector_kwargs=None, distance_range=(0, 100), 
                             points=100):
    if detector_kwargs is None:
        detector_kwargs = {}

    channel = QuantumChannel(ChannelType(channel_type))
    detector = QuantumDetector(**detector_kwargs)
    simulator = simulator_class(channel, detector, distance_km=0)

    results = simulator.simulate_distance_range(
        min_distance=distance_range[0],
        max_distance=distance_range[1],
        num_points=points
    )

    plt.figure(figsize=(10, 6))
    plt.semilogy(results['distances'], results['secret_key_rates'], 'g-', label='SKR (bps)')
    plt.grid(True, which='both')
    plt.xlabel('Distance (km)', fontsize=18)
    plt.ylabel('Secret Key Rate (bps)', fontsize=18)
    plt.title(f'E91: SKR vs Distance ({channel_type.upper()})', fontsize=20)
    plt.legend(fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.show()


In [ ]:
def plot_skr_vs_rate_e91(simulator_class, channel_type="fiber", 
                         rate_values=None, distance_km=50, detector_kwargs=None):
    if detector_kwargs is None:
        detector_kwargs = {}

    if rate_values is None:
        rate_values = np.linspace(0.1e6, 2.0e6, 50)

    skr_values = []

    for rate in rate_values:
        detector = QuantumDetector(pair_generation_rate=rate, **detector_kwargs)
        channel = QuantumChannel(ChannelType(channel_type))
        simulator = simulator_class(channel, detector, distance_km=distance_km)

        result = simulator.simulate_single_distance(distance_km)
        skr_values.append(result['secret_key_rate'])

    plt.figure(figsize=(10, 6))
    plt.plot(rate_values, skr_values, 'c-', label='SKR (bps)')
    plt.grid(True)
    plt.xlabel('Pair Generation Rate (Hz)', fontsize=18)
    plt.ylabel('Secret Key Rate (bps)', fontsize=18)
    plt.title(f'E91: SKR vs Entangled Photon Rate ({channel_type.upper()})', fontsize=20)
    plt.legend(fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.show()
